In [12]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint

In [13]:
df = pd.read_csv('Titanic.csv')

In [14]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
df["label"] = df.Survived
df = df.drop(["PassengerId", "Survived", "Name", "Ticket", "Cabin"], axis=1)

# handling missing values
median_age = df.Age.median()
mode_embarked = df.Embarked.mode()[0]

df = df.fillna({"Age": median_age, "Embarked": mode_embarked})

In [16]:
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,label
0,3,male,22.0,1,0,7.2500,S,0
1,1,female,38.0,1,0,71.2833,C,1
2,3,female,26.0,0,0,7.9250,S,1
3,1,female,35.0,1,0,53.1000,S,1
4,3,male,35.0,0,0,8.0500,S,0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null object
label       891 non-null int64
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [18]:
test_size = 20
def train_test_split(df, test_size):
    if isinstance(test_size, float):
        test_size= round(test_size * len(df))
    indexes = df.index.tolist()
    test_indexes = random.sample(population=indexes , k=test_size)
    test_df = df.loc[test_indexes]
    train_df = df.drop(test_indexes)
    return train_df, test_df


In [19]:
def data_purity(data):
    lable_column = data[ : , -1]
    unique_classes = np.unique(lable_column)
    if len(unique_classes) == 1:
        return True
    else:
        return False

In [20]:
def classify_data(data):
    lable_column = data[ : , -1]
    unique_classes , counts_unique_classes = np.unique(lable_column , return_counts=True)
    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    return classification

In [21]:
def get_potential_splits(data):
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits

In [22]:
def split_data(data, split_column, split_value):
    split_column_values = data[: , split_column]
    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values > split_value]
    
    return data_below, data_above

In [23]:
def calculate_entropy(data):
    label_column=data[: , -1]
    _, counts = np.unique(label_column, return_counts=True)
    
    probabilities = counts / counts.sum()
    
    entropy = sum(probabilities * -np.log2(probabilities))
    
    return entropy

In [24]:
def calculate_overall_entropy(data_below , data_above):
    n_data_points = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n_data_points
    p_data_above = len(data_above) / n_data_points
    
    overall_entropy = (p_data_below * calculate_entropy(data_below) +
                      p_data_above * calculate_entropy(data_above))
    
    
    return overall_entropy

In [25]:
def determine_best_split(data, potential_splits):
    overall_entropy=999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)
            
            if current_overall_entropy < overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    
    
    return best_split_column, best_split_value

In [26]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

In [27]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (data_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # check for empty data
        if len(data_below) == 0 or len(data_above) == 0:
            classification = classify_data(data)
            return classification
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [28]:
tree = decision_tree_algorithm(df,max_depth=3)
pprint(tree)

{'Sex = female': [{'Pclass = 2': [1, {'Fare <= 23.25': [1, 0]}]},
                  {'Fare <= 26.25': [{'Age <= 12.0': [1, 0]}, 0]}]}


In [33]:
tree = decision_tree_algorithm(df,max_depth=10)
pprint(tree)

{'Sex = female': [{'Pclass = 2': [{'Fare <= 28.7125': [{'Fare <= 27.75': [{'Age <= 23.0': [1,
                                                                                           {'Age <= 27.0': [{'Age <= 25.0': [{'Fare <= 13.0': [0,
                                                                                                                                               1]},
                                                                                                                             {'Fare <= 13.8583': [1,
                                                                                                                                                  0]}]},
                                                                                                            {'Age <= 36.0': [1,
                                                                                                                             {'Age <= 38.0': [0,
                                     